In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
def load_clean_data(path):
  df = pd.read_csv(path)
  df.drop(['PassengerId','Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
  df = df.dropna()
  df.Sex = df.Sex.replace({'female': 0, 'male': 1})
  df.Embarked = df.Embarked.replace({'S': 0, 'C':1, 'Q':2})
  return df

In [ ]:
train_data_path = '/content/drive/MyDrive/Colab Notebooks/Domaci2/titanic/train.csv'

df_train = load_clean_data(train_data_path)

y_train = df_train.Survived.to_numpy()
df_train.drop(['Survived'], axis=1, inplace=True)
x_train = df_train.to_numpy()

nb_train = len(x_train)

learning_rate = 0.001
nb_epochs = 100
batch_size = 32

nb_input = 7
nb_hidden1 = 256
nb_hidden2 = 256
nb_classes = 2

w = {
    '1': tf.Variable(tf.random.normal([nb_input, nb_hidden1], dtype=tf.float64)),
    '2': tf.Variable(tf.random.normal([nb_hidden1, nb_hidden2], dtype=tf.float64)),
    'out':tf.Variable(tf.random.normal([nb_hidden2, nb_classes], dtype=tf.float64))
}

b = {
    '1': tf.Variable(tf.random.normal([nb_hidden1], dtype=tf.float64)),
    '2': tf.Variable(tf.random.normal([nb_hidden2], dtype=tf.float64)),
    'out': tf.Variable(tf.random.normal([nb_classes], dtype=tf.float64))
}

f = {
    '1': tf.nn.relu,
    '2': tf.nn.relu,
    'out': tf.nn.softmax
}

def runNN(x):
  z1 = tf.add(tf.matmul(x, w['1']), b['1'])
  a1 = f['1'](z1)
  z2 = tf.add(tf.matmul(a1, w['2']), b['2'])
  a2 = f['2'](z2)
  z_out = tf.add(tf.matmul(a2, w['out']), b['out'])
  out = f['out'](z_out)

  pred = tf.argmax(out, 1)

  return pred, z_out

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

for epoch in range(nb_epochs):
  epoch_loss = 0
  nb_batches = int(nb_train/batch_size)
  for i in range(nb_batches):
    x = x_train[i*batch_size : (i+1)*batch_size, :]
    y = y_train[i*batch_size : (i+1)*batch_size]
    y_onehot = tf.one_hot(y, nb_classes)

    with tf.GradientTape() as tape:
      _, z_out = runNN(x)

      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=z_out, labels=y_onehot))

    w1_g, w2_g, wout_g, b1_g, b2_g, bout_g = tape.gradient(loss, [w['1'], w['2'], w['out'], b['1'], b['2'], b['out']])
    opt.apply_gradients(zip([w1_g, w2_g, wout_g, b1_g, b2_g, bout_g], [w['1'], w['2'], w['out'], b['1'], b['2'], b['out']]))

    epoch_loss += loss

  epoch_loss /= nb_train
  print(f'Epoch: {epoch+1}/{nb_epochs}| Avg loss: {epoch_loss:.5f}')


#Test
test_data_path = '/content/drive/MyDrive/Colab Notebooks/Domaci2/titanic/test.csv'
df_test = load_clean_data(test_data_path)

y_test = df_test.Survived.to_numpy()
df_test.drop(['Survived'], axis=1, inplace=True)
x_test = df_test.to_numpy()

pred, _ = runNN(x_test)
pred_correct = tf.equal(pred, y_test)
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float32))

print(f'Test acc: {accuracy:.3f}')

#Evaluacija modela
predictions = pred
positive_predictions = np.where(predictions == 1)
negative_predictions = np.where(predictions == 0)

TP = np.sum((y_test[positive_predictions] == 1).astype(int))
TN = np.sum((y_test[negative_predictions] == 0).astype(int))
FP = np.sum((y_test[positive_predictions] == 0).astype(int))
FN = np.sum((y_test[negative_predictions] == 1).astype(int))

nb_test = y_test.shape[0]
print('Total test samples: {}'.format(nb_test))
print('TP = {}, TN = {}, FP = {}, FN = {}'.format(TP, TN, FP, FN))


accuracy = (TP + TN) / nb_test
precision = TP / (TP + FP) if TP + FP > 0 else -1
recall = TP / (TP + FN) if TP + FN > 0 else -1
print('A = {:.2f}, P = {:.2f}, R = {:.2f}'.format(accuracy, precision, recall))



Epoch: 1/100| Avg loss: 18.12544
Epoch: 2/100| Avg loss: 11.86954
Epoch: 3/100| Avg loss: 4.87732
Epoch: 4/100| Avg loss: 3.65341
Epoch: 5/100| Avg loss: 2.85218
Epoch: 6/100| Avg loss: 3.03508
Epoch: 7/100| Avg loss: 2.55755
Epoch: 8/100| Avg loss: 2.37131
Epoch: 9/100| Avg loss: 1.24611
Epoch: 10/100| Avg loss: 1.69271
Epoch: 11/100| Avg loss: 2.19549
Epoch: 12/100| Avg loss: 1.66027
Epoch: 13/100| Avg loss: 3.02369
Epoch: 14/100| Avg loss: 0.77118
Epoch: 15/100| Avg loss: 0.92814
Epoch: 16/100| Avg loss: 1.86742
Epoch: 17/100| Avg loss: 2.40074
Epoch: 18/100| Avg loss: 2.63193
Epoch: 19/100| Avg loss: 1.33265
Epoch: 20/100| Avg loss: 1.84754
Epoch: 21/100| Avg loss: 3.27453
Epoch: 22/100| Avg loss: 1.22944
Epoch: 23/100| Avg loss: 2.70242
Epoch: 24/100| Avg loss: 0.98001
Epoch: 25/100| Avg loss: 2.46620
Epoch: 26/100| Avg loss: 1.05824
Epoch: 27/100| Avg loss: 2.03703
Epoch: 28/100| Avg loss: 1.45463
Epoch: 29/100| Avg loss: 3.21853
Epoch: 30/100| Avg loss: 1.21063
Epoch: 31/100| Av